# 3. Estimation of the flux of a variable source, Mrk421

## 3.1 Context
In the first two introductory notebooks, we considered Crab Nebula for its property of being a **bright** and **steady** emitter in the gamma-ray sky. Let us examine now a bright source whose flux is not constant in time: the active galactic nucleus Mrk421.   
We will consider the state of intense gamma-ray activity measured in April 2013 by the MAGIC Collaboration, whose analysis has been published in [MAGIC Collaboration 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..248...29A/abstract).

In [ ]:
# - basic imports (numpy, astropy, regions, matplotlib)
import operator
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.io.fits.verify import VerifyWarning
from regions import PointSkyRegion, CircleSkyRegion
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import logging
import warnings

# - Gammapy's imports
from gammapy.maps import Map, MapAxis, WcsGeom, RegionGeom
from gammapy.data import DataStore, Observation
from gammapy.datasets import SpectrumDataset, Datasets, FluxPointsDataset
from gammapy.makers import (
    SpectrumDatasetMaker,
    WobbleRegionsFinder,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.modeling import Parameter
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    ConstantSpectralModel,
    SkyModel,
    TemporalModel,
    LinearTemporalModel,
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator
from gammapy.stats import WStatCountsStatistic

# - setting up logging and ignoring warnings
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

warnings.filterwarnings("ignore")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", VerifyWarning)

## 3.2. Data Reduction
As before, let us load all the Mrk421 observations and reduce the data.   
Let us focus on the days between the 10th and the 20th of April, where the most intense activity was recorded, by using `Gammapy's` `Observations.select_time` to select these observations.

In [ ]:
# load observations
datastore = DataStore.from_dir("../acme_magic_odas_data/data/Mrk421")
observations = datastore.get_observations(required_irf=["rad_max", "aeff", "edisp"])
print(f"total observations : {len(observations)}")

# select observations between 10th and 20th of April
times = ["2013-04-10T12:00:00", "2013-04-20T12:00:00"]
time_intervals = Time(times, format="isot", scale="utc")
observations = observations.select_time(time_intervals)
print(f"selected observations : {len(observations)}")

As you noticed, we used times from noon to noon. That's just because, as most of the observations happen over night (typically between 20:00 and 06:00, of course depending on the season), that's a safe way of ensuring we are selecting all the observations in a given night.

In [ ]:
# let us define the parameters of the spectrum extraction
# - we need to specify only the center of the on region,
# its radius will be fetched from the RAD_MAX table.
crab_coords = SkyCoord.from_name("Mrk421")
on_region = PointSkyRegion(crab_coords)

# - let us define the energy axes over which we want:
# -- to bin the counts (estimated energies) and
energy_axis = MapAxis.from_energy_bounds(
    10, 1e5, nbin=20, per_decade=False, unit="GeV", name="energy"
)
# -- to interpolate the IRFs (true energies)
energy_axis_true = MapAxis.from_energy_bounds(
    10, 1e5, nbin=28, per_decade=False, unit="GeV", name="energy_true"
)

# let us create an empty dataset with this spatial and energy structure / binning
geom = RegionGeom.create(region=on_region, axes=[energy_axis])
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

In [ ]:
# let us repeat the process of data reduction already illustrated in the previous notebook
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)
# use 3 off regions to estimate the background
region_finder = WobbleRegionsFinder(n_off_regions=3)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

datasets = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(observation.obs_id)), observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    datasets.append(dataset_on_off)

In [ ]:
# let us just take a look at one of the observations to be sure that everything is all right
datasets[0].peek()

## 3.3. Light curve: integral flux vs time

We are now concerned with studying the **evolution of the flux in time**: How could we measure it?      
One option would certainly be to perform the spectrum estimation (demostrated in the previous notebook) per each of the observations (or per day) separately. In this way we could measure spectral changes run by run (or day by day). Let us keep this option in the back of our mind. 

To measure variability in astronomy it is common practice to compile a _light curve_, a graph of the light intensity or brightness as a function of time. To represent the brightness of a source in gamma rays we will consider the **integral flux** above a certain energy, $E_{\rm thr}$

$$
    \phi(E > E_{\rm thr})\,[{\rm cm}^{-2}\,{\rm s}^{-1}] = 
    \int_{E_{\rm thr}}^{\infty} \frac{{\rm d}\phi}{{\rm d}E}(E; \hat{\theta})\,{\rm d}E
$$

where $\hat{\theta}$ represents the parameters of the differential flux model $\frac{{\rm d}\phi}{{\rm d}E}$ we adjusted to the data. $E_{\rm thr}$, the extreme of integration, can depend on the analysis, or on the physics we are interested in. One can in principle perform the likelihood fit of the previous notebook in a given time interval (might be an observation, a day, a week) to determine $\frac{{\rm d}\phi}{{\rm d}E}(E; \hat{\theta})$, and then simply integrate the spectrum obtained.

It is more common practice to fix the parameters of the spectrum adopted $\hat{\theta}$, except for the normalisation $\phi_0$. This is very similar to what we did in the flux point computation. In that case, we re-adjusted the best-fit value for $\phi_0$ using _all the events within an estimated energy bin_. We now re-adjust it using _all the events in a single time interval_.
Given the parallel between the processes of flux points and light curve estimation, `Gammapy` provides a `LightCurveEstimator`  that works very simlarly to the `FluxPointsEstimator`.

Let us then compute the light curve for Mrk421. Which model should we assume for the likelihood fitting? In the paper presenting this dataset ([MAGIC Collaboration, 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..248...29A/abstract)) we read:


> [...] the VHE gamma-ray spectrum from the full nine day data set [...] is well represented by the following log-parabola function:    
> $$ \frac{{\rm d}\phi}{{\rm d}E} = \phi_0 \left( \frac{E}{0.3\,{\rm TeV}} \right)^{- 2.14 - 0.45 \log_{10}(\frac{E}{0.3\,{\rm TeV}})} $$

The amplitude parameter is not specified. Let us see if we can recover the same value fitting all the data together, let us stack and fit them!

In [ ]:
dataset_stacked = datasets.stack_reduce()

# let us use the LP model, use the same reference energy of the paper
total_spectral_model = LogParabolaSpectralModel(
    amplitude=5e-12 * u.Unit("TeV-1 cm-2 s-1"),
    reference=0.3 * u.TeV,
    alpha=2.3 * u.Unit(""),
    beta=0.1 * u.Unit(""),
)
total_model = SkyModel(spectral_model=total_spectral_model, name="Mrk421")

# let us use a reasonable energy range for fitting
e_min = 0.08 * u.TeV
e_max = 10 * u.TeV
dataset_stacked.counts.geom.energy_mask(e_min, e_max)

# assign the model to the
dataset_stacked.models = [total_model]

# run the fit!
fit = Fit()
results = fit.run(datasets=dataset_stacked)
print(results)
print(total_spectral_model)

In [ ]:
print(
    f"alpha = {total_spectral_model.alpha.value:.2f} +/- {total_spectral_model.alpha.error:.2f}"
)
# NOTE: the log in Gammapy's log parabola is in base e
# the one in the paper is in base 10, make the conversion
beta = total_spectral_model.beta.value / np.log10(np.e)
beta_err = total_spectral_model.beta.error / np.log10(np.e)
print(f"beta = {beta:.2f} +/- {beta_err:.2f}")

We, obtained values quite similar to the average spectrum presented in the paper. Let us go ahead and compute the LC with the `LightCurveEstimator`. If no time interval is specified, then one flux point will be estimated for each observation. Let us compute the light curve above 1 TeV, as done in the paper.

In [ ]:
# energy range in which the integral flux has to be computed
energy_edges = [1 * u.TeV, 100 * u.TeV]

# freeze the parameters of the spectral model
# only the norm has to be refitted
total_spectral_model.alpha.frozen = True
total_spectral_model.beta.frozen = True

# rember to assing the model to the datasets
datasets.models = [total_model]

light_curve_estimator_run_wise = LightCurveEstimator(
    energy_edges=energy_edges,
    source="Mrk421",
    reoptimize=False,
    n_sigma_ul=3,
)

light_curve_run_wise = light_curve_estimator_run_wise.run(datasets)

In [ ]:
# let us plot the result
fig, ax = plt.subplots(figsize=(12, 5))
light_curve_run_wise.plot(
    ax=ax, sed_type="flux", marker=".", label=r"run-wise time bins"
)
ax.legend()
ax.set_ylabel(r"$\Phi(E > 1\,{\rm TeV})$")
ax.set_yscale("linear")
plt.show()

It is common practice in astronomy to represent time in [Modified Julian Date (MJD)](https://en.wikipedia.org/wiki/Julian_day) to have a continuous count of time.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
light_curve_run_wise.plot(
    ax=ax,
    sed_type="flux",
    time_format="mjd",
    marker=".",
    label=r"run-wise time bins",
)
ax.legend()
ax.set_ylabel(r"$\Phi(E > 1\,{\rm TeV})\,/\,({\rm cm}^{-2}\,{\rm s}^{-1})$")
ax.set_yscale("linear")
plt.show()

### Exercise 3.1.
Compute the light curve of Mrk 421 assuming a power law spectral model.



## 3.4. Custom time binning
Without specifying any time intervals for the `LightCurveEstimator`, the duration of each run was used for the bins in energy.   
Let us now adopt a custom time binning: let us compute the daily (or, more properly, nightly) integral fluxes. We will use MJD to define the time intervals and again the center of a bin is considered to be at midnight.

In [ ]:
daily_time_intervals = [
    Time([56392.5, 56393.5], format="mjd", scale="utc"),
    Time([56393.5, 56394.5], format="mjd", scale="utc"),
    Time([56394.5, 56395.5], format="mjd", scale="utc"),
    Time([56395.5, 56396.5], format="mjd", scale="utc"),
    Time([56396.5, 56397.5], format="mjd", scale="utc"),
    Time([56397.5, 56398.5], format="mjd", scale="utc"),
    Time([56398.5, 56399.5], format="mjd", scale="utc"),
    Time([56399.5, 56400.5], format="mjd", scale="utc"),
    Time([56400.5, 56401.5], format="mjd", scale="utc"),
]

light_curve_estimator_daily = LightCurveEstimator(
    energy_edges=energy_edges,
    time_intervals=daily_time_intervals,
    source="Mrk421",
    reoptimize=False,
    n_sigma_ul=3,
)
light_curve_daily = light_curve_estimator_daily.run(datasets)

In [ ]:
# let us now plot both the daily and run-wise light curves
fig, ax = plt.subplots(figsize=(12, 5))
light_curve_run_wise.plot(
    ax=ax,
    sed_type="flux",
    time_format="mjd",
    marker=".",
    label="run-wise binning",
    alpha=0.6,
)
light_curve_daily.plot(
    ax=ax,
    sed_type="flux",
    time_format="mjd",
    marker=",",
    label="nightly binning",
    elinewidth=2,
)
ax.set_ylabel(r"$\Phi(E > 1\,{\rm TeV})\,/\,({\rm cm}^{-2}\,{\rm s}^{-1})$")
ax.set_yscale("linear")
ax.set_ylim([0, 4e-10])
plt.show()

## 3.5. Light curve: spectral parameters vs time

For the integral flux estimation we froze all the spectral parameters except for the amplitude. This type of light curve shows us only the overall flux variation, without displaying the change in spectral parameters. It is common practice in high-energy astronomy (whenever we can measure a spectrum) to make a plot of the spectral parameters vs time. Let us use a power-law, for simplicty, and let us compute the daily amplitudes and spectral indexes. We will now use the function `select_time` but for the `Datasets`, to select and fit those in a single day.

In [ ]:
# let us create a list of Spectral Models
spectral_models = []
for i in range(len(daily_time_intervals)):
    _pwl = PowerLawSpectralModel(reference=0.3 * u.TeV)
    spectral_models.append(_pwl)

# define the fitter
fit = Fit()

# let us select a sub-dataset for each day, and
for daily_time_interval, spectral_model in zip(daily_time_intervals, spectral_models):
    # select the dataset in a given time
    daily_datasets = datasets.select_time(*daily_time_interval)
    print(f"{len(daily_datasets)} selected for {daily_time_interval}")
    # apply proper fitting range to all obs. selected
    for _dataset in daily_datasets:
        _dataset.counts.geom.energy_mask(e_min, e_max)
    # define model and assign it
    model = SkyModel(spectral_model=spectral_model, name="Mrk421")
    daily_datasets.models = [model]
    # fit
    results = fit.run(datasets=daily_datasets)

In [ ]:
# now let us make a plot of the spectral parameters
fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True, gridspec_kw={"hspace": 0.05})

# the values of the spectral models should have been automatically updated in the list
# of spectral models
for daily_time_interval, spectral_model in zip(
    daily_time_intervals[1:-1], spectral_models
):
    time_bin_center = 0.5 * (daily_time_interval[0].mjd + daily_time_interval[1].mjd)
    time_bin_width = 0.5 * (daily_time_interval[1].mjd - daily_time_interval[0].mjd)
    ax[0].errorbar(
        time_bin_center - 56392,
        spectral_model.amplitude.value / 1e-9,
        xerr=[time_bin_width],
        yerr=spectral_model.amplitude.error / 1e-9,
        marker=".",
        ls="",
        color="k",
    )
    ax[1].errorbar(
        time_bin_center - 56392,
        spectral_model.index.value,
        xerr=[time_bin_width],
        yerr=spectral_model.index.error,
        marker=".",
        ls="",
        color="k",
    )

ax[0].set_ylabel(
    r"$\phi\,/\,(10^{-9}\,{\rm TeV}^{-1}\,{\rm cm}^{-1}\,{\rm s}^{-1})$", fontsize=10
)
ax[1].set_ylabel(r"$\Gamma$")
ax[1].set_xlabel(r"$({\rm MJD} - 56392)\,/\,{\rm day}$")
plt.show()

## 3.5.1 Harder when brighter

In X-ray as well as gamma-ray astronomy, the "harder when brighter" behavior refers to the phenomenon where the spectrum becomes harder ($< 2$) as the source brightness increases. Observing this type of behaviour gives us relevant information about the acceleration mechanisms of the source. We can check if Mrk 421 shows this behaviour, by making a plot of the integral flux against the spectral index.

In [ ]:
integral_flux_daily = light_curve_daily.flux.data.flatten()
integral_flux_err_daily = light_curve_daily.flux_err.data.flatten()

fig, ax = plt.subplots()

for integral_flux, integral_flux_err, spectral_model in zip(
    integral_flux_daily, integral_flux_err_daily, spectral_models
):
    ax.errorbar(
        integral_flux,
        spectral_model.index.value,
        xerr=[integral_flux_err],
        yerr=[spectral_model.index.error],
        marker=".",
        color="crimson",
    )
plt.gca().invert_yaxis()
ax.set_xlabel(r"$\Phi(E > 1\,{\rm TeV})\,/\,({\rm cm}^{-2}\,{\rm s}^{-1})$")
ax.set_ylabel(r"$\Gamma$")
plt.show()

We indeed observe this correlation.

## 3.6. Light curve: intra-run variability
Going back to the light curve, we can observe that the flare on 13th of April is the one showing not only the highest flux, but also the highest variability. In such cases (very high flux and variability), one might want to check for variability (and thus check the light curve) on even smaller scales than that of a single observational run (typically 20-30 minutes for MAGIC). Let us focus on the 13th of April and let us compute a LC with 5-minute binning.

In [ ]:
flare_day = [
    Time(56394.5, format="mjd", scale="utc"),
    Time(56395.5, format="mjd", scale="utc"),
]
dataset_flare = datasets.select_time(*flare_day)

# let us compute the run-wise LC for just this day
# let us assume a log-parabolic spectrum
dataset_flare.models = [total_model]
# we already have defined a LC estimator with run-wise time binning
light_curve_run_wise_flare = light_curve_estimator_run_wise.run(dataset_flare)

Now let us divide the day in 5 minutes intervals.

**Warning**: in case the time binning is smaller than the run (and hence `Observation` and `Dataset` duration), one must go back to the observations and cut them into smaller times, before creating new data sets, reducing and fitting them. 

In [ ]:
# running this cell can take a few moments
duration = 5 * u.min

# let us split the day in 10 minutes interval
time_intervals_5min = [flare_day[0]]

while time_intervals_5min[-1] <= flare_day[1]:
    time_intervals_5min.append(time_intervals_5min[-1] + duration)

time_intervals_5min = [
    Time([tstart, tstop])
    for tstart, tstop in zip(time_intervals_5min[:-1], time_intervals_5min[1:])
]

# and now cut the observations in these time intervals
short_observations = observations.select_time(time_intervals_5min)
# check that observations have been filtered
print(f"observations after time filtering: {len(short_observations)}")
print(short_observations[1].gti)

In [ ]:
# make datasets out of the new observations
short_datasets = Datasets()

for observation in short_observations:
    dataset = dataset_maker.run(dataset_empty.copy(), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    short_datasets.append(dataset_on_off)

In [ ]:
# use the log parabola model and compute the LC
short_datasets.models = [total_model]

light_curve_estimator_5min = LightCurveEstimator(
    energy_edges=energy_edges,
    time_intervals=time_intervals_5min,
    source="Mrk421",
    reoptimize=False,
    n_sigma_ul=3,
)
light_curve_5min_flare = light_curve_estimator_5min.run(short_datasets)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
light_curve_run_wise_flare.plot(
    ax=ax,
    sed_type="flux",
    time_format="mjd",
    marker=".",
    label="run-wise binning",
    alpha=0.6,
)
light_curve_5min_flare.plot(
    ax=ax,
    sed_type="flux",
    time_format="mjd",
    marker=".",
    label="5-min binning",
)
# ax.set_xlim([56394.8, 56395.2])
ax.set_yscale("linear")
ax.set_ylim([0, 5e-10])
plt.show()

**Note**: this is just an illustrative example. Be very careful on how you bin your LC, deciding on the optimal binning is a delicate task. 

## 3.7. Light curve: fitting the light curve

Let us now use `Gammapy`'s temporal models to fit the flux points of the LC. Let us start with a simple linear model.

In [ ]:
# Create the datasets by iterating over the returned lightcurve
lc_fp_dataset = FluxPointsDataset(data=light_curve_run_wise_flare, name="dataset_lc")

In [ ]:
# let us use, define the temporal model, midnight of that day is t_ref
linear_time_model = LinearTemporalModel(alpha=2, beta=5 / u.d, t_ref=56395 * u.d)
linear_time_model.alpha.frozen = True
# let also add a constant spectral model and let us fit
spectral_model = ConstantSpectralModel(const=1e-10 * u.Unit("TeV-1 cm-2 s-1"))

lc_model = SkyModel(
    spectral_model=spectral_model,
    temporal_model=linear_time_model,
    name="time_model",
)

lc_fp_dataset.models = lc_model
print(lc_fp_dataset)

In [ ]:
fit = Fit()
result = fit.run(lc_fp_dataset)
print(results)
display(result.parameters.to_table())

In [ ]:
lc_fp_dataset.plot_spectrum(axis_name="time")
plt.yscale("linear")

Let us define a custom model to add a Gaussian to try to reproduce the higher bump in the flare.

In [ ]:
class GaussianPlusLinearTemporalModel(TemporalModel):
    """A Gaussian temporal model superimposed to a linear one"""

    tag = ["GaussianPlusLinearTemporalModel", "gauss_lin"]

    _t_ref_default = Time("2000-01-01")
    t_ref = Parameter("t_ref", _t_ref_default.mjd, unit="day", frozen=False)
    # line parameters
    alpha = Parameter("alpha", 1.0, frozen=False)
    beta = Parameter("beta", 0.0, unit="d-1", frozen=False)
    # Gaussian parameters
    amplitude = Parameter("amplitude", 1.0, frozen=False)
    sigma = Parameter("sigma", "2 h", frozen=False)

    @staticmethod
    def evaluate(time, t_ref, alpha, beta, amplitude, sigma):
        line = alpha + beta * (time - t_ref)
        gauss = amplitude * np.exp(-((time - t_ref) ** 2) / (2 * sigma**2))
        return line + gauss

In [ ]:
# let us assume the reference is the peak of the flare
t_peak = Time("2013-04-13T02:20:00")
gauss_line_temporal_model = GaussianPlusLinearTemporalModel(
    t_ref=t_peak.mjd * u.d, sigma=0.3 * u.h, alpha=2.5, amplitude=2, beta=5 / u.d
)
gauss_line_temporal_model.t_ref.frozen = True
gauss_line_temporal_model.alpha.frozen = True

lc_model2 = SkyModel(
    spectral_model=spectral_model,
    temporal_model=gauss_line_temporal_model,
    name="time_model2",
)
# assign this new model to the flux points of the LC
lc_fp_dataset.models = lc_model2

# check the initial model before fitting
lc_fp_dataset.plot_spectrum(axis_name="time")
plt.yscale("linear")

In [ ]:
fit = Fit()
result = fit.run(lc_fp_dataset)
print(results)
display(result.parameters.to_table())

In [ ]:
lc_fp_dataset.plot_spectrum(axis_name="time")
plt.yscale("linear")

### Exercise 3.2
Estimate the goodness of this fit. 